In [ ]:
# MOVE THIS TO SOME KIND OF TEST ELSEWHERE.

T_op = SparseIntegralOp(
    6, 2, 5, 2.0,
    'elasticT3', [1.0, 0.25], m.pts, m.tris, np.float32,
    farfield_op_type = FMMFarfieldBuilder(150, 3.0, 450)
)
A_op = SparseIntegralOp(
    6, 2, 5, 2.0,
    'elasticA3', [1.0, 0.25], m.pts, m.tris, np.float32,
    farfield_op_type = FMMFarfieldBuilder(150, 3.0, 450)
)
# A is the transpose of T and mass_op is its own transpose.
iop_T = SumOp([A_op, mass_op])

A_near = sum([op.to_bsr().to_scipy() for op in A_op.nearfield.mat_no_correction])
T_near = sum([op.to_bsr().to_scipy() for op in T_op.nearfield.mat_no_correction])

A_near = A_near.tocsr()
T_near = T_near.tocsr()

np.set_printoptions(linewidth = 200)
a, b = 0, 14
rattt = np.round(A_near[a:b,a:b].todense() / T_near[a:b,a:b].T.todense(),1)
small = np.where(np.abs(rattt) != 1)
rattt, T_near[a:b,a:b].todense(), A_near[a:b,a:b].todense(), A_near[small], T_near.T[small]

A_near[0:6,0:6].todense() - T_near.T[0:6,0:6].todense()

which = np.zeros((100,100))
which[np.where(np.abs((A_near[:100,:100] - T_near.T[:100,:100]).todense()) > 4e-2)] = 1
which2 = np.zeros((100,100))
which2[np.where(np.abs(A_near[:100,:100].todense()) > 1e-3)] = 1
plt.figure(figsize = (10, 10))
plt.subplot(121)
plt.spy(which)
plt.subplot(122)
plt.spy(which2)
plt.show()